In [1]:
%tensorflow_version 2.x
import tensorflow as tf

TensorFlow 2.x selected.


In [2]:
#Load data
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
sample, sample_label = x_train[0], y_train[0]
x_train.shape

11493376/11490434 [==============================] - 0s 0us/step


(60000, 28, 28)

# ANN Autoencoder

In [5]:
activation = 'relu'
autoencoder_input = tf.keras.Input(shape=(28, 28), name='original_img')
x = tf.keras.layers.Flatten()(autoencoder_input)
x = tf.keras.layers.Dense(784, activation=activation)(x)
x = tf.keras.layers.Dense(512, activation=activation)(x)
x = tf.keras.layers.Dense(256, activation=activation)(x)
x = tf.keras.layers.Dense(512, activation=activation)(x)
x = tf.keras.layers.Dense(784, activation=activation)(x)
autoencoder_output = tf.keras.layers.Reshape((28, 28))(x)

autoencoder = tf.keras.Model(autoencoder_input, autoencoder_output, name='autoencoder')
autoencoder.summary()

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
original_img (InputLayer)    [(None, 28, 28)]          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 784)               615440    
_________________________________________________________________
dense_6 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_8 (Dense)              (None, 512)               131584    
_________________________________________________________________
dense_9 (Dense)              (None, 784)               

In [6]:
autoencoder.compile(tf.keras.optimizers.Adam(learning_rate=1e-3), loss=tf.keras.losses.MeanSquaredError())
callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]
autoencoder.fit(x_train, x_train, epochs=50, validation_split=0.2, batch_size=32, callbacks=callback, shuffle=True)

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 7s 137us/sample - loss: 0.0161 - val_loss: 0.0116
Epoch 2/50
48000/48000 [==============================] - 5s 94us/sample - loss: 0.0110 - val_loss: 0.0104
Epoch 3/50
48000/48000 [==============================] - 5s 95us/sample - loss: 0.0101 - val_loss: 0.0098
Epoch 4/50
48000/48000 [==============================] - 4s 94us/sample - loss: 0.0096 - val_loss: 0.0095
Epoch 5/50
48000/48000 [==============================] - 5s 95us/sample - loss: 0.0093 - val_loss: 0.0093
Epoch 6/50
48000/48000 [==============================] - 5s 94us/sample - loss: 0.0090 - val_loss: 0.0090
Epoch 7/50
48000/48000 [==============================] - 5s 95us/sample - loss: 0.0089 - val_loss: 0.0089
Epoch 8/50
48000/48000 [==============================] - 5s 96us/sample - loss: 0.0087 - val_loss: 0.0089
Epoch 9/50
48000/48000 [==============================] - 5s 94us/sample - loss: 0.0082 - val

In [7]:
# Prediction on training
autoencoder.evaluate(x_train, x_train, verbose=2)

60000/60000 - 3s - loss: 0.0073


0.0073259227309376

In [8]:
# Prediction on test
autoencoder.evaluate(x_test,  x_test, verbose=2)

10000/10000 - 1s - loss: 0.0073


0.007263591511547565

# Test ANN Autoencoder

In [0]:
auc_x_train = autoencoder.predict(x_train)
auc_x_test = autoencoder.predict(x_test)

In [0]:
tf.random.set_seed(0)

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]
model.fit(auc_x_train, y_train, epochs=100, validation_split=0.2, callbacks=callback)

Train on 48000 samples, validate on 12000 samples
Epoch 1/100
48000/48000 [==============================] - 4s 88us/sample - loss: 0.3435 - accuracy: 0.8999 - val_loss: 0.1654 - val_accuracy: 0.9528
Epoch 2/100
48000/48000 [==============================] - 4s 83us/sample - loss: 0.1673 - accuracy: 0.9493 - val_loss: 0.1191 - val_accuracy: 0.9643
Epoch 3/100
48000/48000 [==============================] - 4s 81us/sample - loss: 0.1281 - accuracy: 0.9612 - val_loss: 0.1018 - val_accuracy: 0.9691
Epoch 4/100
48000/48000 [==============================] - 4s 82us/sample - loss: 0.1061 - accuracy: 0.9685 - val_loss: 0.0972 - val_accuracy: 0.9718
Epoch 5/100
48000/48000 [==============================] - 4s 82us/sample - loss: 0.0936 - accuracy: 0.9705 - val_loss: 0.0877 - val_accuracy: 0.9741
Epoch 6/100
48000/48000 [==============================] - 4s 83us/sample - loss: 0.0807 - accuracy: 0.9746 - val_loss: 0.0861 - val_accuracy: 0.9743
Epoch 7/100
48000/48000 [=========================

In [12]:
model.evaluate(auc_x_train,  y_train, verbose=2)

60000/60000 - 3s - loss: 0.0311 - accuracy: 0.9911


[0.031133462774875807, 0.99113333]

In [13]:
model.evaluate(auc_x_test,  y_test, verbose=2)

10000/10000 - 1s - loss: 0.0868 - accuracy: 0.9783


[0.08678828507277503, 0.9783]